In [1]:
import billboard
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import base64
from requests import post, get
import json
import time
from datetime import datetime, timedelta

from client_info import CLIENT_ID
from client_info import CLIENT_SECRET

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
data_df = pd.read_csv("data_df.csv")

# Display sample data
data_df.head()

,Song Name,Artist,Spotify ID,Genres,Duration,Popularity,Release Date
0,Bad Day,Daniel Powter,0mUyMawtxj1CJ76kn9gIZK,NaN,3:53,71,2005-02-22
1,Temperature,Sean Paul,0k2GOhqsrxDTAbFFSdNJjT,dancehall,3:38,78,2005-09-27
2,Promiscuous,"Nelly Furtado, Timbaland",2gam98EZKrF9XuOkU13ApN,NaN,4:02,83,2006-01-01
3,You're Beautiful,James Blunt,0vg4WnUWvze6pBOJDTq99k,NaN,3:29,73,2005-08-08
4,Hips Don't Lie (feat. Wyclef Jean),"Shakira, Wyclef Jean",3ZFTkvIE7kyPt6Nu3PEa7V,"latin pop, latin",3:38,84,2005-11-28


In [5]:
data_df["Year"] = ""
data_df.head()

,Song Name,Artist,Spotify ID,Genres,Duration,Popularity,Release Date,Year
0,Bad Day,Daniel Powter,0mUyMawtxj1CJ76kn9gIZK,NaN,3:53,71,2005-02-22,
1,Temperature,Sean Paul,0k2GOhqsrxDTAbFFSdNJjT,dancehall,3:38,78,2005-09-27,
2,Promiscuous,"Nelly Furtado, Timbaland",2gam98EZKrF9XuOkU13ApN,NaN,4:02,83,2006-01-01,
3,You're Beautiful,James Blunt,0vg4WnUWvze6pBOJDTq99k,NaN,3:29,73,2005-08-08,
4,Hips Don't Lie (feat. Wyclef Jean),"Shakira, Wyclef Jean",3ZFTkvIE7kyPt6Nu3PEa7V,"latin pop, latin",3:38,84,2005-11-28,


In [7]:
# Convert 'Year' column to numeric values, coercing errors to NaN
data_df['Year'] = pd.to_numeric(data_df['Year'], errors='coerce')

# Fill NaN values (if any) with 0 (or any other placeholder you want)
data_df['Year'].fillna(0, inplace=True)

# Convert 'Year' back to integers after filling NaN
data_df['Year'] = data_df['Year'].astype(int)

# Add 2006 to the 'Year' column for the first 100 rows
data_df.loc[:99, 'Year'] = 2006

# Add 1 to the 'Year' for every subsequent block of 100 rows (e.g., 2007, 2008, etc.)
for i in range(1, (len(data_df) // 100) + 1):
    start_idx = i * 100
    end_idx = (i + 1) * 100
    data_df.loc[start_idx:end_idx - 1, 'Year'] = 2006 + i

# Output the result
print(data_df[['Song Name', 'Year']].head(110))  # Display first 110 rows to check

                              Song Name  Year
0                               Bad Day  2006
1                           Temperature  2006
2                           Promiscuous  2006
3                      You're Beautiful  2006
4    Hips Don't Lie (feat. Wyclef Jean)  2006
..                                  ...   ...
105                    Before He Cheats  2007
106                   Hey There Delilah  2007
107                    I Wanna Love You  2007
108                        Say It Right  2007
109                           Glamorous  2007

[110 rows x 2 columns]


C:\Users\duvoe\AppData\Local\Temp\ipykernel_18244\2953739787.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_df['Year'].fillna(0, inplace=True)


In [9]:
data_df.head(1900)

,Song Name,Artist,Spotify ID,Genres,Duration,Popularity,Release Date,Year
0,Bad Day,Daniel Powter,0mUyMawtxj1CJ76kn9gIZK,NaN,3:53,71,2005-02-22,2006
1,Temperature,Sean Paul,0k2GOhqsrxDTAbFFSdNJjT,dancehall,3:38,78,2005-09-27,2006
2,Promiscuous,"Nelly Furtado, Timbaland",2gam98EZKrF9XuOkU13ApN,NaN,4:02,83,2006-01-01,2006
3,You're Beautiful,James Blunt,0vg4WnUWvze6pBOJDTq99k,NaN,3:29,73,2005-08-08,2006
4,Hips Don't Lie (feat. Wyclef Jean),"Shakira, Wyclef Jean",3ZFTkvIE7kyPt6Nu3PEa7V,"latin pop, latin",3:38,84,2005-11-28,2006
...,...,...,...,...,...,...,...,...
1895,Bulletproof,La Roux,3kMrazSvILsgcwtidZd1Qd,NaN,3:25,71,2009,2024
1896,FE!N (feat. Playboi Carti),"Travis Scott, Playboi Carti",3KEMjT3QVfKIl4E9DIcrmF,rap,3:11,70,2023-07-26,2024
1897,The Painter,Cody Johnson,6UoKX6uLJwhsnyTp5k5StP,"texas country, country, red dirt",3:31,64,2023-08-11,2024
1898,Down Bad,Taylor Swift,5O4SPACkaBSKjBJEPYJwfQ,NaN,4:21,50,2024-04-19,2024


In [11]:
data_df.drop('Release Date', axis=1, inplace=True)
data_df.head()

,Song Name,Artist,Spotify ID,Genres,Duration,Popularity,Year
0,Bad Day,Daniel Powter,0mUyMawtxj1CJ76kn9gIZK,NaN,3:53,71,2006
1,Temperature,Sean Paul,0k2GOhqsrxDTAbFFSdNJjT,dancehall,3:38,78,2006
2,Promiscuous,"Nelly Furtado, Timbaland",2gam98EZKrF9XuOkU13ApN,NaN,4:02,83,2006
3,You're Beautiful,James Blunt,0vg4WnUWvze6pBOJDTq99k,NaN,3:29,73,2006
4,Hips Don't Lie (feat. Wyclef Jean),"Shakira, Wyclef Jean",3ZFTkvIE7kyPt6Nu3PEa7V,"latin pop, latin",3:38,84,2006
